In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 200)

### Importer les données en format CSV

In [2]:
donnees_brutes = pd.read_excel('../Data/Originaux/XLS_Centris_par_municipalite_fromNumbers.xlsx', index_col=0)

In [3]:
donnees = donnees_brutes.astype(str).reset_index()
donnees.columns = donnees.columns.str.lstrip('Unnamed: ')
donnees = donnees.replace({'nan': ''})
donnees = donnees.dropna(how='all', axis=1)
donnees = donnees.iloc[: , :-29]

donnees = donnees.replace('\n',' ', regex=True)
donnees = donnees.replace('  ',' ', regex=True)
donnees = donnees.replace('- ','-', regex=True)
donnees = donnees.replace(' -','-', regex=True)

### Trouver à quelle municipalité / quartier appartient la rangée

In [4]:
# Trouver les termes récurrents de la colonne de gauche pour les retirer, ce qui permettra d'identifier les noms propres
# termes_recurrents = donnees['Janvier - mars 2022'].value_counts()[:35].index.tolist() --> j'ai retiré Total de la liste originale

termes_recurrents = ['Unifamiliale', 'Coprop./App.', 'Com./Ind./Ent.', 'Terre/Terrain', 'Prop. à revenus', 'Ferme/Fermette', 'Maison de plain-pied', 'Maison à étages', 'Terrain', 'Commerciale', 'Maison à un étage et demi', 'Janvier - mars 2022', 'Duplex', 'Maison à paliers multiples', 'Triplex', 'Terre', 'Autre', 'Appartement', 'Quadruplex', 'Maison mobile', 'Fermette', 'Maison', 'Entreprise (fonds de commerce)', 'Quintuplex', 'Industrielle', 'Ferme', 'Loft/Studio', 'Inconnu', 'commerce)', 'Entreprise (fonds de']

sans_termes_recurrents = donnees[~donnees['Janvier - mars 2022'].isin(termes_recurrents)]

In [5]:
# Certaines cellules ont été mal analysées par Adobe et contiennent des espaces + nombres. Il faut donc utiliser startswith pour les retirer

sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Unités', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Répartition', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Écart', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Territoires', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Autre', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Grand t', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Maison', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Com./Ind./Ent.', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Prop.', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('653', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('États-Unis', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Ferme ', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Fermette ', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Ferme/Fermette ', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Terre  ', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Terrain  ', na=False)]
sans_termes_recurrents = sans_termes_recurrents[~sans_termes_recurrents['Janvier - mars 2022'].str.startswith('Terre/Terrain  ', na=False)]

In [6]:
# Retirer ensuite toutes les rangées vides

sans_termes_recurrents = sans_termes_recurrents[sans_termes_recurrents['Janvier - mars 2022'].notna()]

### QUARTIERS

In [7]:
# Lorsqu'une municipalité est sous-divisée par quartiers, chacun d'entre eux commencent par "Quartier", ce qui me permet de les identifier facilement

sans_termes_recurrents[['Municipalite', 'Quartier']] = sans_termes_recurrents['Janvier - mars 2022'].str.split('Quartier', 1, expand=True)
sans_termes_recurrents['Quartier'] = sans_termes_recurrents['Quartier'].str.lstrip()
sans_termes_recurrents['Quartier'] = sans_termes_recurrents['Quartier'].fillna('Aucun')

In [8]:
# Remplir la colonne 

sans_termes_recurrents['Municipalite'] = sans_termes_recurrents['Municipalite'].replace(r'^\s*$', np.nan, regex=True)
muni_quartiers_merge = sans_termes_recurrents[['Municipalite','Quartier']]
muni_quartiers_merge['Municipalite'] = muni_quartiers_merge['Municipalite'].replace({'Janvier-mars 2022': np.nan})

muni_quartiers_merge.head(45)

<ipython-input-8-23e251acc6df>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muni_quartiers_merge['Municipalite'] = muni_quartiers_merge['Municipalite'].replace({'Janvier-mars 2022': np.nan})


,Municipalite,Quartier
1,Amos,Aucun
19,NaN,Aucun
21,NaN,Amos-Est
37,NaN,Amos-Ouest
53,NaN,Aucun
55,NaN,Centre-Ville
67,NaN,Domaine Bellevue
77,NaN,Aucun
79,Barraute,Aucun
98,Berry,Aucun


### Gérer les municipalités qui ont le même nom

In [9]:
duplicates_muni = ['Clermont', 'Rivière-Ojima', 'Saint-Lambert', 'Sainte-Félicité', 'Saint-Damase', 'Saint-Léon-le-Grand', 'Saint-Donat', "Sainte-Jeanne-d'Arc", 'Sainte-Françoise', 'Saint-Cyprien', 'Saint-Siméon', "L'Ange-Gardien",'Notre-Dame-de-Lourdes', 'Sainte-Monique', 'Sainte-Perpétue', 'Saint-Elzéar', 'Saint-Isidore', 'Sainte-Sabine', 'Saint-Louis-de-Gonzague', 'Saint-Séverin', 'Saint-Augustin', 'Rivière-Saint-Jean', 'Saint-Sébastien', 'Saint-Stanislas']

In [10]:
# Puisque certaines municipalités ont le même nom et que les régions ne font pas partie de mes données, je dois les distinguer en les ajoutant moi-même à la main

sans_termes_recurrents[sans_termes_recurrents['Municipalite'].isin(duplicates_muni)][['Municipalite', 'Quartier']]

,Municipalite,Quartier
389,Clermont,Aucun
612,Rivière-Ojima,Aucun
672,Saint-Lambert,Aucun
2664,Sainte-Félicité,Aucun
2901,Saint-Damase,Aucun
2940,Saint-Léon-le-Grand,Aucun
3290,Saint-Donat,Aucun
3334,Sainte-Jeanne-d'Arc,Aucun
3535,Sainte-Françoise,Aucun
4256,Saint-Cyprien,Aucun


In [11]:
muni_quartiers_merge["Region"] = ''

muni_quartiers_merge.at[389, 'Region'] = 'Abitibi-Témiscamingue'
muni_quartiers_merge.at[612, 'Region'] = 'Abitibi-Témiscamingue'
muni_quartiers_merge.at[672, 'Region'] = 'Abitibi-Témiscamingue'
muni_quartiers_merge.at[2664, 'Region'] = 'Bas-Saint-Laurent'
muni_quartiers_merge.at[2901, 'Region'] = 'Bas-Saint-Laurent'
muni_quartiers_merge.at[2940, 'Region'] = 'Bas-Saint-Laurent'
muni_quartiers_merge.at[3290, 'Region'] = 'Bas-Saint-Laurent'
muni_quartiers_merge.at[3334, 'Region'] = 'Bas-Saint-Laurent'
muni_quartiers_merge.at[3535, 'Region'] = 'Bas-Saint-Laurent'
muni_quartiers_merge.at[4256, 'Region'] = 'Bas-Saint-Laurent'
muni_quartiers_merge.at[5203, 'Region'] = 'Capitale-Nationale'
muni_quartiers_merge.at[5313, 'Region'] = 'Capitale-Nationale' 
muni_quartiers_merge.at[5471, 'Region'] = 'Capitale-Nationale' 
muni_quartiers_merge.at[8712, 'Region'] = 'Centre-du-Québec' 
muni_quartiers_merge.at[9361, 'Region'] = 'Centre-du-Québec' 
muni_quartiers_merge.at[9702, 'Region'] = 'Centre-du-Québec' 
muni_quartiers_merge.at[9716, 'Region'] = 'Centre-du-Québec' 
muni_quartiers_merge.at[10942, 'Region'] = 'Chaudière-Appalaches'
muni_quartiers_merge.at[10994, 'Region'] = 'Chaudière-Appalaches'
muni_quartiers_merge.at[11686, 'Region'] = 'Chaudière-Appalaches'
muni_quartiers_merge.at[11748, 'Region'] = 'Chaudière-Appalaches'
muni_quartiers_merge.at[11759, 'Region'] = 'Chaudière-Appalaches'
muni_quartiers_merge.at[12356, 'Region'] = 'Chaudière-Appalaches'
muni_quartiers_merge.at[12380, 'Region'] = 'Chaudière-Appalaches'
muni_quartiers_merge.at[13275, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[13785, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[14176, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[14938, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[15824, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[17966, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[17990, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[18317, 'Region'] = 'Côte-Nord'
muni_quartiers_merge.at[19564, 'Region'] = 'Lanaudière'
muni_quartiers_merge.at[20798, 'Region'] = 'Lanaudière'
muni_quartiers_merge.at[26283, 'Region'] = 'Mauricie'
muni_quartiers_merge.at[26578, 'Region'] = 'Mauricie'
muni_quartiers_merge.at[26891, 'Region'] = 'Mauricie'
muni_quartiers_merge.at[28155, 'Region'] = 'Montérégie'
muni_quartiers_merge.at[29881, 'Region'] = 'Estrie'
muni_quartiers_merge.at[30599, 'Region'] = 'Montérégie'
muni_quartiers_merge.at[32676, 'Region'] = 'Montérégie'
muni_quartiers_merge.at[33132, 'Region'] = 'Montérégie'
muni_quartiers_merge.at[39907, 'Region'] = 'Outaouais'
muni_quartiers_merge.at[41419, 'Region'] = 'Saguenay–Lac-Saint-Jean'
muni_quartiers_merge.at[42405, 'Region'] = 'Saguenay-Lac-Saint-Jean'
muni_quartiers_merge.at[42424, 'Region'] = 'Saguenay-Lac-Saint-Jean'
muni_quartiers_merge.at[42448, 'Region'] = 'Saguenay-Lac-Saint-Jean'

<ipython-input-11-7b524f5c6c8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muni_quartiers_merge["Region"] = ''


### Rassembler les données sur les municipalités / quartiers avec les données d'origine

In [12]:
donnees_avec_mun = pd.merge(donnees, muni_quartiers_merge, left_index=True, right_index=True, how='outer')
donnees_avec_mun = donnees_avec_mun[donnees_avec_mun.columns.drop(list(donnees_avec_mun.filter(regex='_y')))]
donnees_avec_mun.columns = donnees_avec_mun.columns.str.rstrip('_x')

print(donnees.shape)
print(donnees_avec_mun.shape)

(43533, 48)
(43533, 51)


In [13]:
# Je m'assure ensuite de remplir les municipalités et quartiers par le dernier en haut

donnees_avec_mun['Municipalite'] = donnees_avec_mun['Municipalite'].fillna(method='ffill')
donnees_avec_mun['Quartier'] = donnees_avec_mun['Quartier'].fillna(method='ffill')
donnees_avec_mun['Region'] = donnees_avec_mun['Region'].fillna(method='ffill')
donnees_avec_mun[['Municipalite', 'Quartier', 'Region']].head(35)

,Municipalite,Quartier,Region
0,NaN,NaN,NaN
1,Amos,Aucun,
2,Amos,Aucun,
3,Amos,Aucun,
4,Amos,Aucun,
5,Amos,Aucun,
6,Amos,Aucun,
7,Amos,Aucun,
8,Amos,Aucun,
9,Amos,Aucun,


### Retirer les rangées inutiles

In [14]:
donnees_sans_detail = donnees_avec_mun[~donnees_avec_mun['Janvier - mars 2022'].str.startswith('Maison', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Entreprise', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Répartition', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Unités', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Écart', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Autre', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('653', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Loft', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Terre ', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Terrain ', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Territoires', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Ferme ', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].str.startswith('Fermette', na=False)]

donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Janvier - mars 2022'].isnull()]


In [15]:
categories_retirer = ['Appartement', 'Duplex', 'Triplex', 'Quadruplex', 'Quintuplex', 'Terre', 'Terrain', 'Commerciale', 'Entreprise', 'Industrielle', 'Inconnu', 'États-Unis / USA', 'Janvier-mars 2022', 'commerce)']

for mot in categories_retirer:
    donnees_sans_detail = donnees_sans_detail.drop(donnees_sans_detail[donnees_sans_detail['Janvier - mars 2022'] == mot].index)

### Nettoyage additionnel

In [16]:
donnees_sans_detail[['Type_propriete', 'Donnees_extra']] = donnees_sans_detail['Janvier - mars 2022'].str.split('                             ', 1, expand=True)
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Municipalite'].str.startswith('Janvier', na=False)]
donnees_sans_detail = donnees_sans_detail[~donnees_sans_detail['Municipalite'].str.startswith('Total', na=False)]

donnees_sans_detail['Type_propriete'] = np.where((donnees_sans_detail['Municipalite'] == donnees_sans_detail['Type_propriete']) | (donnees_sans_detail['Type_propriete'].str.startswith('Quartier')), 'Total_proprietes', donnees_sans_detail['Type_propriete'])

In [17]:
donnees_sans_detail = donnees_sans_detail.replace({'Métabetchouan/L ac-à-la-Croix': 'Métabetchouan/Lac-à-la-Croix', 'Montréal (Mercier/Hochela ga-Maisonneuve)': 'Montréal (Mercier/Hochelaga-Maisonneuve)'})

### Concatenate les chiffres dispersés à travers plusieurs colonnes

In [18]:
donnees_sans_detail['Nouv_inscriptions'] = donnees_sans_detail['1'] + donnees_sans_detail['2'] + donnees_sans_detail['3']
donnees_sans_detail['Inscript_vigueur'] = donnees_sans_detail['5'] + donnees_sans_detail['6'] + donnees_sans_detail['7']
donnees_sans_detail['Nombre_ventes'] = donnees_sans_detail['8'] + donnees_sans_detail['9'] + donnees_sans_detail['10']
donnees_sans_detail['Volume_ventes'] = donnees_sans_detail['13'] + donnees_sans_detail['14'] + donnees_sans_detail['15'] + donnees_sans_detail['16'] + donnees_sans_detail['17'] + donnees_sans_detail['18'] + donnees_sans_detail['19'] + donnees_sans_detail['20']
donnees_sans_detail['Delai'] = donnees_sans_detail['23']

In [19]:
donnees_sans_detail['Nouv_inscriptions'] = donnees_sans_detail['Nouv_inscriptions'].str.replace(' ', '')
donnees_sans_detail['Inscript_vigueur'] = donnees_sans_detail['Inscript_vigueur'].str.replace(' ', '')
donnees_sans_detail['Nombre_ventes'] = donnees_sans_detail['Nombre_ventes'].str.replace(' ', '')
donnees_sans_detail['Volume_ventes'] = donnees_sans_detail['Volume_ventes'].str.replace(' ', '')
donnees_sans_detail['Delai'] = donnees_sans_detail['Delai'].str.replace(' ', '')

In [20]:
donnees_sans_detail['Prix_vente_moyen1'] = donnees_sans_detail['29'] + donnees_sans_detail['30'] + donnees_sans_detail['31'] + donnees_sans_detail['32'] + donnees_sans_detail['33']
donnees_sans_detail['Prix_vs_inscription1'] = donnees_sans_detail['41'] + donnees_sans_detail['42'] + donnees_sans_detail['43']
donnees_sans_detail['Prix_vs_evaluation1'] = donnees_sans_detail['45'] + donnees_sans_detail['46'] + donnees_sans_detail['47']

### Prix de vente vs inscription et évaluation

In [21]:
# Certaines cellules n'ont pas été séparées par Adobe, je dois trouver lesquelles et le faire moi-même

donnees_sans_detail['Prix_vente_moyen'] = donnees_sans_detail['Prix_vente_moyen1'].str.split('  ').str[0]
donnees_sans_detail['Prix_vs_inscription2'] = donnees_sans_detail['Prix_vente_moyen1'].str.split('  ').str[4]
donnees_sans_detail['Prix_vs_evaluation2'] = donnees_sans_detail['Prix_vente_moyen1'].str.split('  ').str[6]

In [22]:
donnees_sans_detail['Prix_vs_inscription1'] = donnees_sans_detail['Prix_vs_inscription1'].replace(r'^\s*$', np.nan, regex=True)
donnees_sans_detail['Prix_vs_inscription2'] = donnees_sans_detail['Prix_vs_inscription2'].replace(r'^\s*$', np.nan, regex=True)
donnees_sans_detail['Prix_vs_evaluation1'] = donnees_sans_detail['Prix_vs_evaluation1'].replace(r'^\s*$', np.nan, regex=True)
donnees_sans_detail['Prix_vs_evaluation2'] = donnees_sans_detail['Prix_vs_evaluation2'].replace(r'^\s*$', np.nan, regex=True)

In [23]:
# Les positions des espaces ne fonctionnent pas quand les prix de vente sont > 1M$, je dois refaire l'exercice en changeant l'indexation

donnees_sans_detail['Prix_vs_inscription3'] = np.where((donnees_sans_detail['Prix_vs_inscription1'].isnull()) & (donnees_sans_detail['Prix_vs_inscription2'].isnull()) & (donnees_sans_detail['Nombre_ventes'] != 0), donnees_sans_detail['Prix_vente_moyen1'].str.split('  ').str[-3], donnees_sans_detail['Prix_vs_inscription2'])
donnees_sans_detail['Prix_vs_evaluation3'] = np.where((donnees_sans_detail['Prix_vs_evaluation1'].isnull()) & (donnees_sans_detail['Prix_vs_evaluation2'].isnull()) & (donnees_sans_detail['Nombre_ventes'] != 0), donnees_sans_detail['Prix_vente_moyen1'].str.split('  ').str[-1], donnees_sans_detail['Prix_vs_evaluation2'])

In [24]:
donnees_sans_detail['Prix_vs_inscription'] = np.where(donnees_sans_detail['Prix_vs_inscription3'].isnull(), donnees_sans_detail['Prix_vs_inscription1'], donnees_sans_detail['Prix_vs_inscription3'])
donnees_sans_detail['Prix_vs_evaluation'] = np.where(donnees_sans_detail['Prix_vs_evaluation3'].isnull(), donnees_sans_detail['Prix_vs_evaluation1'], donnees_sans_detail['Prix_vs_evaluation3'])

In [25]:
donnees_sans_detail['Prix_vente_moyen'] = donnees_sans_detail['Prix_vente_moyen'].str.replace(' ', '')
donnees_sans_detail['Prix_vs_inscription'] = donnees_sans_detail['Prix_vs_inscription'].str.replace(' ', '')
donnees_sans_detail['Prix_vs_evaluation'] = donnees_sans_detail['Prix_vs_evaluation'].str.replace(' ', '')

### Créer nouveau df avec les colonnes importantes

In [26]:
final_data = donnees_sans_detail[['Region', 'Municipalite', 'Quartier', 'Type_propriete', 'Nouv_inscriptions', 'Nombre_ventes', 'Volume_ventes', 'Delai', 'Prix_vente_moyen', 'Prix_vs_inscription', 'Prix_vs_evaluation']]
final_data = final_data.reset_index()

In [27]:
# Puisque j'ai certaines rangées avec une valeur 'Région' (pour handle mes duplicates) , je me crée une nouvelle colonne qui me servira de string unique selon la géographie pour faciliter le merge plus tard

final_data['muni_regions_norm'] = np.where(final_data['Region'] != '', final_data['Region'] + final_data['Municipalite'] + final_data['Quartier'], '')
final_data['muni_regions_norm'] = final_data["muni_regions_norm"].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')

final_data['muni_regions_norm'] = final_data['muni_regions_norm'].str.lower()
final_data['muni_regions_norm'] = final_data['muni_regions_norm'].str.replace('(', '')
final_data['muni_regions_norm'] = final_data['muni_regions_norm'].str.replace(')', '')
final_data['muni_regions_norm'] = final_data['muni_regions_norm'].str.replace('aucun', '')
final_data['muni_regions_norm'] = final_data['muni_regions_norm'].str.replace(' ', '')

<ipython-input-27-eeed9b538024>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  final_data['muni_regions_norm'] = final_data['muni_regions_norm'].str.replace('(', '')
<ipython-input-27-eeed9b538024>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  final_data['muni_regions_norm'] = final_data['muni_regions_norm'].str.replace(')', '')


In [28]:
# Je fais ensuite la même chose dans une nouvelle colonne pour ceux qui n'ont pas de région (tous sauf les duplicates)

final_data['muni_quartiers_norm'] = np.where(final_data['muni_regions_norm'] == '', final_data['Municipalite'] + final_data['Quartier'], '')
final_data['muni_quartiers_norm'] = final_data["muni_quartiers_norm"].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')

final_data['muni_quartiers_norm'] = final_data['muni_quartiers_norm'].str.lower()
final_data['muni_quartiers_norm'] = final_data['muni_quartiers_norm'].str.replace('(', '')
final_data['muni_quartiers_norm'] = final_data['muni_quartiers_norm'].str.replace(')', '')
final_data['muni_quartiers_norm'] = final_data['muni_quartiers_norm'].str.replace('aucun', '')
final_data['muni_quartiers_norm'] = final_data['muni_quartiers_norm'].str.replace(' ', '')

final_data.head(45)

<ipython-input-28-b7a957ae44a6>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  final_data['muni_quartiers_norm'] = final_data['muni_quartiers_norm'].str.replace('(', '')
<ipython-input-28-b7a957ae44a6>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  final_data['muni_quartiers_norm'] = final_data['muni_quartiers_norm'].str.replace(')', '')


,index,Region,Municipalite,Quartier,Type_propriete,Nouv_inscriptions,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,muni_regions_norm,muni_quartiers_norm
0,1,,Amos,Aucun,Total_proprietes,31,35,8604750,205,,NaN,,,amos
1,2,,Amos,Aucun,Unifamiliale,27,25,7161000,82,286440,98,124,,amos
2,8,,Amos,Aucun,Coprop./App.,0,0,0,0,0,0,0,,amos
3,10,,Amos,Aucun,Prop. à revenus,3,4,1031500,133,,NaN,,,amos
4,14,,Amos,Aucun,Ferme/Fermette,0,0,0,0,,NaN,,,amos
5,15,,Amos,Aucun,Terre/Terrain,0,5,232650,911,,NaN,,,amos
6,17,,Amos,Aucun,Com./Ind./Ent.,1,1,179600,32,,NaN,,,amos
7,21,,Amos,Amos-Est,Total_proprietes,11,13,3899500,96,,NaN,,,amosamos-est
8,22,,Amos,Amos-Est,Unifamiliale,11,10,3227000,57,322700,99,124,,amosamos-est
9,27,,Amos,Amos-Est,Coprop./App.,0,0,0,0,0,0,0,,amosamos-est


In [29]:
final_data_avec_region = final_data[final_data['muni_regions_norm'] != '']
final_data_sans_region = final_data[final_data['muni_regions_norm'] == '']
final_data_sans_region = final_data_sans_region[~final_data_sans_region['Municipalite'].isin(duplicates_muni)]

In [30]:
print(final_data_avec_region.shape)
print(final_data_sans_region.shape)

(356, 14)
(13079, 14)


### Importer & matcher municipalités propres

In [31]:
match_municipalites = pd.read_csv('../Data/muni_quartiers_propres.csv')
match_municipalites['Arrondissement / sous-municipalité'] = match_municipalites['Arrondissement / sous-municipalité'].fillna('')
match_municipalites['Quartiers'] = match_municipalites['Quartiers'].fillna('')

In [32]:
# Je me crée de nouveau un string unique selon la géographie (avec région)

match_municipalites['muni_regions_norm'] = match_municipalites['Région'] + match_municipalites['Municipalités'] + match_municipalites['Arrondissement / sous-municipalité'] + match_municipalites['Quartiers']
match_municipalites['muni_regions_norm'] = match_municipalites["muni_regions_norm"].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')

match_municipalites['muni_regions_norm'] = match_municipalites['muni_regions_norm'].str.lower()
match_municipalites['muni_regions_norm'] = match_municipalites['muni_regions_norm'].str.replace('(', '')
match_municipalites['muni_regions_norm'] = match_municipalites['muni_regions_norm'].str.replace(')', '')
match_municipalites['muni_regions_norm'] = match_municipalites['muni_regions_norm'].str.replace('aucun', '')
match_municipalites['muni_regions_norm'] = match_municipalites['muni_regions_norm'].str.replace(' ', '')

<ipython-input-32-9869454f4a5d>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  match_municipalites['muni_regions_norm'] = match_municipalites['muni_regions_norm'].str.replace('(', '')
<ipython-input-32-9869454f4a5d>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  match_municipalites['muni_regions_norm'] = match_municipalites['muni_regions_norm'].str.replace(')', '')


In [33]:
# Je crée la colonne avec un string unique sans région pour matcher les autres pour ensuite joindre tout le monde

match_municipalites['muni_quartiers_norm'] = match_municipalites['Municipalités'] + match_municipalites['Arrondissement / sous-municipalité'] + match_municipalites['Quartiers']
match_municipalites['muni_quartiers_norm'] = match_municipalites["muni_quartiers_norm"].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')

match_municipalites['muni_quartiers_norm'] = match_municipalites['muni_quartiers_norm'].str.lower()
match_municipalites['muni_quartiers_norm'] = match_municipalites['muni_quartiers_norm'].str.replace('(', '')
match_municipalites['muni_quartiers_norm'] = match_municipalites['muni_quartiers_norm'].str.replace(')', '')
match_municipalites['muni_quartiers_norm'] = match_municipalites['muni_quartiers_norm'].str.replace('aucun', '')
match_municipalites['muni_quartiers_norm'] = match_municipalites['muni_quartiers_norm'].str.replace(' ', '')

<ipython-input-33-b2170e5f46be>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  match_municipalites['muni_quartiers_norm'] = match_municipalites['muni_quartiers_norm'].str.replace('(', '')
<ipython-input-33-b2170e5f46be>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  match_municipalites['muni_quartiers_norm'] = match_municipalites['muni_quartiers_norm'].str.replace(')', '')


In [34]:
matched1 = pd.merge(final_data_avec_region, match_municipalites, on=['muni_regions_norm'], how='inner')
matched1

,index,Region,Municipalite,Quartier,Type_propriete,Nouv_inscriptions,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,muni_regions_norm,muni_quartiers_norm_x,Géographie,Région,Municipalités,Arrondissement / sous-municipalité,Quartiers,muni_quartiers_norm_y
0,389,Abitibi-Témiscamingue,Clermont,Aucun,Total_proprietes,0,0,0,0,,NaN,,abitibi-temiscamingueclermont,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont
1,390,Abitibi-Témiscamingue,Clermont,Aucun,Unifamiliale,0,0,0,0,0,0,0,abitibi-temiscamingueclermont,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont
2,393,Abitibi-Témiscamingue,Clermont,Aucun,Coprop./App.,0,0,0,0,0,0,0,abitibi-temiscamingueclermont,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont
3,394,Abitibi-Témiscamingue,Clermont,Aucun,Prop. à revenus,0,0,0,0,,NaN,,abitibi-temiscamingueclermont,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont
4,395,Abitibi-Témiscamingue,Clermont,Aucun,Ferme/Fermette,0,0,0,0,,NaN,,abitibi-temiscamingueclermont,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,41424,Saguenay–Lac-Saint-Jean,Sainte-Monique,Aucun,Coprop./App.,0,0,0,0,0,0,0,saguenaylac-saint-jeansainte-monique,,Municipalité,Saguenay–Lac-Saint-Jean,Sainte-Monique,,,sainte-monique
294,41425,Saguenay–Lac-Saint-Jean,Sainte-Monique,Aucun,Prop. à revenus,0,0,0,0,,NaN,,saguenaylac-saint-jeansainte-monique,,Municipalité,Saguenay–Lac-Saint-Jean,Sainte-Monique,,,sainte-monique
295,41426,Saguenay–Lac-Saint-Jean,Sainte-Monique,Aucun,Ferme/Fermette,0,0,0,0,,NaN,,saguenaylac-saint-jeansainte-monique,,Municipalité,Saguenay–Lac-Saint-Jean,Sainte-Monique,,,sainte-monique
296,41427,Saguenay–Lac-Saint-Jean,Sainte-Monique,Aucun,Terre/Terrain,0,0,0,0,,NaN,,saguenaylac-saint-jeansainte-monique,,Municipalité,Saguenay–Lac-Saint-Jean,Sainte-Monique,,,sainte-monique


In [35]:
matched2 = pd.merge(final_data_sans_region, match_municipalites, on='muni_quartiers_norm', how='left')
matched2

,index,Region,Municipalite,Quartier,Type_propriete,Nouv_inscriptions,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,muni_regions_norm_x,muni_quartiers_norm,Géographie,Région,Municipalités,Arrondissement / sous-municipalité,Quartiers,muni_regions_norm_y
0,1,,Amos,Aucun,Total_proprietes,31,35,8604750,205,,NaN,,,amos,Municipalité,Abitibi-Témiscamingue,Amos,,,abitibi-temiscamingueamos
1,2,,Amos,Aucun,Unifamiliale,27,25,7161000,82,286440,98,124,,amos,Municipalité,Abitibi-Témiscamingue,Amos,,,abitibi-temiscamingueamos
2,8,,Amos,Aucun,Coprop./App.,0,0,0,0,0,0,0,,amos,Municipalité,Abitibi-Témiscamingue,Amos,,,abitibi-temiscamingueamos
3,10,,Amos,Aucun,Prop. à revenus,3,4,1031500,133,,NaN,,,amos,Municipalité,Abitibi-Témiscamingue,Amos,,,abitibi-temiscamingueamos
4,14,,Amos,Aucun,Ferme/Fermette,0,0,0,0,,NaN,,,amos,Municipalité,Abitibi-Témiscamingue,Amos,,,abitibi-temiscamingueamos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13081,42893,,Saguenay (Shipshaw),Aucun,Coprop./App.,0,0,0,0,0,0,0,,saguenayshipshaw,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,saguenaylac-saint-jeansaguenayshipshaw
13082,42894,,Saguenay (Shipshaw),Aucun,Prop. à revenus,0,0,0,0,,NaN,,,saguenayshipshaw,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,saguenaylac-saint-jeansaguenayshipshaw
13083,42895,,Saguenay (Shipshaw),Aucun,Ferme/Fermette,0,0,0,0,,NaN,,,saguenayshipshaw,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,saguenaylac-saint-jeansaguenayshipshaw
13084,42896,,Saguenay (Shipshaw),Aucun,Terre/Terrain,3,2,301200,16,,NaN,,,saguenayshipshaw,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,saguenaylac-saint-jeansaguenayshipshaw


In [36]:
matched3 = pd.concat([matched1, matched2]).drop(columns=['index', 'muni_regions_norm_y', 'muni_regions_norm', 'muni_quartiers_norm'])
matched3 = matched3.drop_duplicates()
matched3

,Region,Municipalite,Quartier,Type_propriete,Nouv_inscriptions,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,muni_quartiers_norm_x,Géographie,Région,Municipalités,Arrondissement / sous-municipalité,Quartiers,muni_quartiers_norm_y,muni_regions_norm_x
0,Abitibi-Témiscamingue,Clermont,Aucun,Total_proprietes,0,0,0,0,,NaN,,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont,NaN
1,Abitibi-Témiscamingue,Clermont,Aucun,Unifamiliale,0,0,0,0,0,0,0,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont,NaN
2,Abitibi-Témiscamingue,Clermont,Aucun,Coprop./App.,0,0,0,0,0,0,0,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont,NaN
3,Abitibi-Témiscamingue,Clermont,Aucun,Prop. à revenus,0,0,0,0,,NaN,,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont,NaN
4,Abitibi-Témiscamingue,Clermont,Aucun,Ferme/Fermette,0,0,0,0,,NaN,,,Municipalité,Abitibi-Témiscamingue,Clermont,,,clermont,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13081,,Saguenay (Shipshaw),Aucun,Coprop./App.,0,0,0,0,0,0,0,NaN,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,NaN,
13082,,Saguenay (Shipshaw),Aucun,Prop. à revenus,0,0,0,0,,NaN,,NaN,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,NaN,
13083,,Saguenay (Shipshaw),Aucun,Ferme/Fermette,0,0,0,0,,NaN,,NaN,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,NaN,
13084,,Saguenay (Shipshaw),Aucun,Terre/Terrain,3,2,301200,16,,NaN,,NaN,Municipalité,Saguenay–Lac-Saint-Jean,Saguenay,Shipshaw,,NaN,


In [37]:
print(final_data.shape)
print(matched1.shape)
print(matched2.shape)
print(matched3.shape)

(13435, 14)
(298, 20)
(13086, 20)
(13359, 19)


In [38]:
test1 = final_data[['Municipalite', 'Quartier', 'Nombre_ventes']]
test2 = matched3[['Municipalite', 'Quartier', 'Nombre_ventes']]

pd.concat([test1, test2]).drop_duplicates(keep=False)

,Municipalite,Quartier,Nombre_ventes
6075,Saint-Siméon,Aucun,1


### Trouver les duplicates

In [39]:
duplicates = matched3[matched3.duplicated(['Région', 'Municipalite', 'Quartier', 'Type_propriete'], keep=False)]
duplicates['Nombre_ventes'] = duplicates['Nombre_ventes'].str.replace(' ', '')
duplicates['Nombre_ventes'] = duplicates['Nombre_ventes'].astype(float)
duplicates[duplicates['Nombre_ventes'] > 4].head(60)

<ipython-input-39-1be7aa1997db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates['Nombre_ventes'] = duplicates['Nombre_ventes'].str.replace(' ', '')
<ipython-input-39-1be7aa1997db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates['Nombre_ventes'] = duplicates['Nombre_ventes'].astype(float)


,Region,Municipalite,Quartier,Type_propriete,Nouv_inscriptions,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,muni_quartiers_norm_x,Géographie,Région,Municipalités,Arrondissement / sous-municipalité,Quartiers,muni_quartiers_norm_y,muni_regions_norm_x


### Exporter les données pour analyse

In [40]:
donnees_propres = matched3[['Région', 'Municipalite', 'Quartier', 'Type_propriete', 'Nombre_ventes', 'Volume_ventes', 'Delai', 'Prix_vente_moyen', 'Prix_vs_inscription', 'Prix_vs_evaluation', 'Géographie']]

In [41]:
donnees_propres.at[10828, 'Prix_vs_inscription'] = 101
donnees_propres.at[10828, 'Prix_vs_evaluation'] = 165
donnees_propres.at[10829, 'Prix_vs_evaluation'] = 152

/Users/lauriannecroteau/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/lauriannecroteau/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/lauriannecroteau/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [42]:
donnees_propres = donnees_propres.drop_duplicates()
donnees_propres.to_csv('../Data/donnees_propres.csv')

donnees_propres.sort_values(by='Prix_vente_moyen', ascending=False)

,Région,Municipalite,Quartier,Type_propriete,Nombre_ventes,Volume_ventes,Delai,Prix_vente_moyen,Prix_vs_inscription,Prix_vs_evaluation,Géographie
11003,Montréal,Montréal (Rosemont/La Petite-Patrie),Petite Italie,Unifamiliale,1,999000,36,999000,100,0,Quartier
11123,Montréal,Montréal (Ville-Marie),Cité du Havre,Coprop./App.,3,2975000,105,991667,95,153,Quartier
7576,Laurentides,Mirabel,Mirabel en Haut,Unifamiliale,11,10890900,45,990082,100,176,Quartier
1233,Bas-Saint-Laurent,Les Hauteurs,Aucun,Unifamiliale,1,99000,148,99000,99,154,Municipalité
1559,Bas-Saint-Laurent,Saint-Marcellin,Aucun,Unifamiliale,1,99000,70,99000,83,0,Municipalité
...,...,...,...,...,...,...,...,...,...,...,...
4954,Estrie,Saint-Venant-de-Paquette,Aucun,Com./Ind./Ent.,0,0,0,,NaN,,Municipalité
4955,Estrie,Stanstead-Est,Aucun,Total_proprietes,1,302500,48,,NaN,,Municipalité
4958,Estrie,Stanstead-Est,Aucun,Prop. à revenus,0,0,0,,NaN,,Municipalité
4959,Estrie,Stanstead-Est,Aucun,Ferme/Fermette,0,0,0,,NaN,,Municipalité
